In [2]:
import pandas as pd
import numpy as np
import gc
import random
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.cross_validation import StratifiedKFold, cross_val_score, KFold
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from lightgbm import sklearn as lgbmsk
from lightgbm import LGBMRegressor, LGBMClassifier

In [27]:
from sklearn.model_selection import StratifiedKFold

In [6]:
from utils.stacking import StackingClassifier

## Import Data

In [32]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

y_train = train.target
id_test = test['id']

## Preprocess data

In [33]:

#train['ps_ind_sum_bin'] = train[BIN].sum(axis = 1 )
#train['ps_reg_mult'] = train.ps_reg_01 * train.ps_reg_02 * train.ps_reg_03
#train['ps_car_13_ps_reg_03']= train.ps_car_13* train.ps_reg_03
train['ps_car_15'] = (train.ps_car_15)**2
train['ps_car_14'] = (train.ps_car_14)**2
train['ps_car_12'] = round((train.ps_car_12)**2,4) * 10000
train['ps_car_13'] = (train.ps_car_13)**2 * 48400
train['ps_reg_03'] = (4*train.ps_reg_03)**2

#test['ps_ind_sum_bin'] = test[BIN].sum(axis = 1 )
#test['ps_reg_mult'] = test.ps_reg_01 * test.ps_reg_02 * test.ps_reg_03
#test['ps_car_13_ps_reg_03']= test.ps_car_13* test.ps_reg_03
test['ps_car_15'] = (test.ps_car_15)**2
test['ps_car_14'] = (test.ps_car_14)**2
test['ps_car_12'] = round((test.ps_car_12)**2,4) * 10000
test['ps_car_13'] = (test.ps_car_13)**2 * 48400
test['ps_reg_03'] = (4*test.ps_reg_03)**2

In [34]:
drop_columns = ['id', 'ps_calc_10', 'ps_calc_01', 'ps_calc_04', 'ps_car_02_cat', 'ps_calc_14',
               'ps_calc_08', 'ps_calc_17_bin', 'ps_car_10_cat', 'ps_ind_11_bin', 'ps_calc_12',
               'ps_calc_09', 'ps_car_06_cat', 'ps_calc_05','ps_calc_16_bin', 'ps_calc_20_bin',
                'ps_calc_18_bin']

train.drop(drop_columns, axis = 1, inplace = True)
train.drop('target', axis = 1, inplace = True)
test.drop(drop_columns, axis = 1, inplace = True)

In [35]:
CAT = []
for col in train.columns:
    if 'cat' in col:
        CAT.append(col)
        
BIN = []
for col in train.columns:
    if 'bin' in col:
        BIN.append(col)
        
CALC = []
for col in train.columns:
    if 'calc' in col:
        CALC.append(col)

data = pd.concat((train, test), axis=0, ignore_index=True)
for col in CAT:
    data = pd.concat((data, pd.get_dummies(data[col], prefix=col)), axis=1)
    data.drop(col, axis=1, inplace=True)

train = data.iloc[:train.shape[0],:]
test = data.iloc[train.shape[0]:,:]

test.index = range(len(test))

In [44]:
clfs = {#ExtraTreesClassifier(n_estimators=800, criterion='gini', max_depth=37, max_features=25, 
                             #min_samples_split=4, min_samples_leaf=2, n_jobs=-1, random_state=888),                  
        'xgb': XGBClassifier(n_estimators=600, learning_rate=0.03, max_depth=10, colsample_bytree=0.4, 
                      min_child_weight=1, seed=88888), 
        'random forest' : RandomForestClassifier(n_estimators=800, criterion='gini'),
        'lgb' : LGBMClassifier(learning_rate= 0.01,n_estimators= 1250, max_bin= 10, subsample= 0.8, 
                      subsample_freq = 10, colsample_bytree = 0.8, max_depth = 10)}

In [45]:
meta_model = LogisticRegression(C=1.0, penalty='l2', n_jobs=-1)

In [46]:
stacking_model = StackingClassifier(clfs, meta_model= meta_model, cv= StratifiedKFold(n_splits=5, random_state=115, shuffle=True))

In [47]:
stacking_model.fit(train, y_train)

KeyboardInterrupt: 